In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px

df = pd.read_csv("/kaggle/input/enrichednytimescovid19/covid19_us_county.csv")
pos_df = pd.read_csv("/kaggle/input/enrichednytimescovid19/us_county_pop_and_shps.csv")

df.head()

,date,county,state,fips,state_fips,county_fips,cases,deaths,new_day_cases,new_day_deaths,cases_per_capita_100k,deaths_per_capita_100k,new_day_cases_per_capita_100k,new_day_deaths_per_capita_100k,county_pop_2019_est,pop_per_sq_mile_2010
0,2020-01-21,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
1,2020-01-22,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
2,2020-01-23,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8
3,2020-01-24,Cook,Illinois,17031,17,31,1,0,0.0,0.0,0.019417,0.0,0.0,0.0,5150233.0,5495.1
4,2020-01-24,Snohomish,Washington,53061,53,61,1,0,0.0,0.0,0.121642,0.0,0.0,0.0,822083.0,341.8


In [12]:
# What do we need for df? - Cases in each region per population in Arizona
p = df.groupby(["state",'county','county_pop_2019_est'])['cases'].sum()
q = df.groupby(['state','county'])['county_pop_2019_est'].mean()
# print(p.head())
# Now we have cases per region and population, time to specify Arizona
p = p['Arizona']
print(p)
# print(p[0])
q = q['Arizona']
q.head()

# Go through each element in q and replace it with p[i] / q[i]
for i in range(len(q)):
    q[i] = p[i]/q[i]
q

MultiIndex([(    'Apache',   71887.0),
            (   'Cochise',  125922.0),
            (  'Coconino',  143476.0),
            (      'Gila',   54018.0),
            (    'Graham',   38837.0),
            (  'Greenlee',    9498.0),
            (    'La Paz',   21108.0),
            (  'Maricopa', 4485414.0),
            (    'Mohave',  212181.0),
            (    'Navajo',  110924.0),
            (      'Pima', 1047279.0),
            (     'Pinal',  462789.0),
            ('Santa Cruz',   46498.0),
            (   'Yavapai',  235099.0),
            (      'Yuma',  213787.0)],
           names=['county', 'county_pop_2019_est'])


/tmp/ipykernel_30/2795623616.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_30/2795623616.py:14: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`



county
Apache        0.784676
Cochise       0.031305
Coconino      0.356847
Gila          0.026139
Graham        0.034426
Greenlee      0.026742
La Paz        0.158897
Maricopa      0.103146
Mohave        0.072820
Navajo        0.718654
Pima          0.112806
Pinal         0.090078
Santa Cruz    0.266485
Yavapai       0.056189
Yuma          0.188688
Name: county_pop_2019_est, dtype: float64

In [21]:
L = []
for i in range(len(p.index)):
    L.append(p.index[i][1])
L

[71887.0,
 125922.0,
 143476.0,
 54018.0,
 38837.0,
 9498.0,
 21108.0,
 4485414.0,
 212181.0,
 110924.0,
 1047279.0,
 462789.0,
 46498.0,
 235099.0,
 213787.0]

In [3]:
# What do we need for pos_df? - only the center long/lat
pos = pos_df.drop(['fips','county_pop_2019_est', 'center_point', 'county_geom'], axis = 1)

# Gets the lat and long positional values for arizona 
pos_lat = pos.groupby(['state','county'])['county_center_lat'].mean()
print(pos_lat['Arizona'])
pos_lon = pos.groupby(['state','county'])['county_center_lon'].mean()
print(pos_lon['Arizona'])

county
Apache        35.394866
Cochise       31.890358
Coconino      35.841451
Gila          33.805895
Graham        32.940967
Greenlee      33.227090
La Paz        33.733410
Maricopa      33.359637
Mohave        35.704264
Navajo        35.401640
Pima          32.104763
Pinal         32.914284
Santa Cruz    31.529969
Yavapai       34.607195
Yuma          32.773686
Name: county_center_lat, dtype: float64
county
Apache       -109.486870
Cochise      -109.750719
Coconino     -111.759932
Gila         -110.807391
Graham       -109.888830
Greenlee     -109.243637
La Paz       -113.978255
Maricopa     -112.488494
Mohave       -113.749618
Navajo       -110.314143
Pima         -111.790885
Pinal        -111.348376
Santa Cruz   -110.846130
Yavapai      -112.546517
Yuma         -113.909604
Name: county_center_lon, dtype: float64


In [51]:
fd = pd.DataFrame({'county': q.index, 'pop':L,'val' : q,'lat': pos_lat['Arizona'], 'lon': pos_lon['Arizona']})
fd

,county,pop,val,lat,lon
county,,,,,
Apache,Apache,71887.0,0.784676,35.394866,-109.486870
Cochise,Cochise,125922.0,0.031305,31.890358,-109.750719
Coconino,Coconino,143476.0,0.356847,35.841451,-111.759932
Gila,Gila,54018.0,0.026139,33.805895,-110.807391
Graham,Graham,38837.0,0.034426,32.940967,-109.888830
Greenlee,Greenlee,9498.0,0.026742,33.227090,-109.243637
La Paz,La Paz,21108.0,0.158897,33.733410,-113.978255
Maricopa,Maricopa,4485414.0,0.103146,33.359637,-112.488494
Mohave,Mohave,212181.0,0.072820,35.704264,-113.749618


In [52]:
fig = px.scatter_mapbox(
    fd,
    lat = 'lat', lon = 'lon',
    zoom = 5, 
    hover_name = 'county',
    hover_data = 'pop',
    size='val',
    mapbox_style = 'carto-positron'
)
fig.show()